In [1]:
from symmer.chemistry import xyz_from_pubchem
import numpy as np
from openfermion import FermionOperator
from scipy.sparse.linalg import expm
from openfermion import get_sparse_operator
from symmer.symplectic import PauliwordOp, QuantumState
import itertools
from scipy.sparse import csr_matrix, coo_matrix
from scipy.sparse.linalg import eigsh
from pyscf import ci

In [41]:
mol_name = 'H2O'
# mol_name = 'Be'
# mol_name = 'LiH'

In [42]:
xyz_file = xyz_from_pubchem(mol_name)
print(xyz_file)

3
 
O	0	0	0
H	0.2774	0.8929	0.2544
H	0.6068	-0.2383	-0.7169



In [43]:
from symmer.chemistry import Draw_molecule

viewer = Draw_molecule(xyz_file,
                       width=400,
                       height=400,
                       style="stick") # change to sphere/stick
viewer.show()

## can move around in notebook!

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [44]:
from symmer.chemistry import PySCFDriver

In [45]:
basis = 'STO-3G'
convergence = 1e-6
charge=0
max_hf_cycles=50
ram = 8_000
run_mp2  = False,
run_cisd = True,
run_ccsd = False,
run_fci  = False

In [46]:
pyscf_obj = PySCFDriver(xyz_file,
                       basis,
                       convergence=convergence,
                       charge=charge,
                       max_ram_memory=ram,
                       max_hf_cycles=max_hf_cycles,
                       
                       run_mp2=run_mp2,
                       run_cisd=run_cisd,
                       run_ccsd=run_ccsd,
                       run_fci=run_fci)

In [47]:
pyscf_obj.run_pyscf()

In [48]:
2*pyscf_obj.pyscf_hf.mol.nao

14

In [49]:
from symmer.chemistry import FermionicHamiltonian
from openfermion import jordan_wigner, get_sparse_operator

In [50]:
H_ferm = FermionicHamiltonian(pyscf_obj.pyscf_hf)

H_ferm.build_fermionic_hamiltonian_operator()

H = jordan_wigner(H_ferm.fermionic_molecular_hamiltonian)

In [51]:
H_symmer = PauliwordOp.from_openfermion(H)

In [52]:
hf_state = H_ferm.hf_fermionic_basis_state
hf_state


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0])

In [53]:
H_cisd = H_ferm.get_cisd_fermionic()

In [54]:
E_CISD, vec_CISD = eigsh(H_cisd, which='SA', k=1)
cisd_qstate = QuantumState.from_array(vec_CISD)
del vec_CISD
E_CISD += pyscf_obj.pyscf_hf.mol.energy_nuc() #<- need to add back in nuclear energy!

In [55]:
pyscf_obj.pyscf_cisd.e_tot

-75.01477057556038

In [56]:
E_CISD - pyscf_obj.pyscf_cisd.e_tot

array([-1.23634436e-11])

In [66]:
cisd_qstate.dagger * H_symmer * cisd_qstate

(-75.01477057557257+0j)

In [ ]:
class CI_from_qubit_H():
    def __init__(self, qubitH, hf_array_number):
        self.H = qubitH
        self.hf_array = hf_array_number # HF in number basis (not BK / parity / ...)
        self.n_electrons = np.sum(self.hf_array)
        
    def _gen_single_excitations(self):
        single_dets = []
        single_excitations = []
        for i in range(self.n_electrons):
            for a in range(self.n_electrons, self.H.n_qubits):
                single_excitations.append((i, a))

                det = self.hf_array.copy()
                det[[i,a]] = det[[a,i]]
                single_dets.append(det)
        return single_dets#, single_excitations
    
    def _gen_double_excitations(self):
        double_dets = []
        double_excitations = []
        for i in range(self.n_electrons):
            for j in range(i+1, self.n_electrons):
                for a in range(self.n_electrons, self.H.n_qubits):
                    for b in range(a+1, self.H.n_qubits):
                        double_excitations.append((i,j, a,b))

                        det = self.hf_array.copy()
                        det[[i,a]] = det[[a,i]]
                        det[[j,b]] = det[[b,j]]
                        double_dets.append(det)
        return double_dets#, double_excitations
                        
    def _gen_single_double_excitations(self):
        double_dets = []
        double_excitations = []
        single_dets = []
        single_excitations = []
        for i in range(self.n_electrons):
            for a in range(self.n_electrons, self.H.n_qubits):
                single_excitations.append((i, a))

                det = self.hf_array.copy()
                det[[i,a]] = det[[a,i]]
                single_dets.append(det)
                for j in range(i+1, self.n_electrons):
                    for b in range(a+1, self.H.n_qubits):
                        double_excitations.append((i,j, a,b))

                        det = self.hf_array.copy()
                        det[[i,a]] = det[[a,i]]
                        det[[j,b]] = det[[b,j]]
                        double_dets.append(det)
                        
        #return single_dets, single_excitations, double_dets, double_excitations
        return [*single_dets, *double_dets]
    
    def _perform_CI_JW(self, det_list):
        
        # include HF array
        det_list = [self.hf_array, *det_list]
        
        data =[]
        row =[]
        col =[]
        for i, det_i in enumerate(det_list):
            for det_j in det_list:
                index_i = int(''.join(det_i.astype(str)),2)
                q_state_i = QuantumState(np.array(det_i).reshape([1,-1]), [1])
                    
                index_j = int(''.join(det_j.astype(str)),2)
                q_state_j = QuantumState(np.array(det_j).astype(int).reshape([1,-1]), [1])
                
                mat_ij_element = q_state_j.dagger * self.H * q_state_i
                data.append(mat_ij_element)
                row.append(index_i)
                col.append(index_j)
                
#                 # ij == ji ! (symmetry)
#                 data.append(mat_ij_element)
#                 row.append(index_j)
#                 col.append(index_i)
                
        H_CI_JW = csr_matrix((data, (row, col)), shape=(2**self.H.n_qubits, 2**self.H.n_qubits))
#         H_CI_JW = csr_matrix((data, (row, col)))
        return H_CI_JW
    
    def perform_CI(self, method='CISD', encoding='JW'):
        if method== 'CISD':
            det_list = self._gen_single_double_excitations()
        elif method== 'CIS':
            det_list = self._gen_single_excitations()
        elif method== 'CID':
            det_list = self._gen_double_excitations()
        else:
            raise ValueError(f'unknown / not implemented CI method: {method}')
            
        if encoding=='JW':
            H_CI = self._perform_CI_JW(det_list)
        elif encoding=='BK':
            H_CI = self._perform_CI_BK(det_list)
        
#         print(H_CI.shape)
        E_CI, vec_CI = eigsh(H_CI, which='SA', k=1)
        ci_qstate = QuantumState.from_array(vec_CI)
        del vec_CI
        return H_CI, E_CI, ci_qstate
    def _perform_CI_BW(self, det_list):
        pass
    

In [ ]:
HF_qstate = QuantumState(hf_state.reshape([1,-1]), [1])
print(pyscf_obj.pyscf_hf.e_tot)
HF_qstate.dagger * H_symmer * HF_qstate

In [ ]:
# n_occ = H_ferm.n_electrons
# n_virt = H_ferm.n_qubits-H_ferm.n_electrons

# HF_term = 1
# single_terms =  n_occ* n_virt
# double_terms = (n_occ**2-n_occ)/2 *  (n_virt**2-n_virt)/2

# total = HF_term + single_terms + double_terms
# total

In [ ]:
hf_state = H_ferm.hf_fermionic_basis_state

test = CI_from_qubit_H(H_symmer, hf_state)

In [ ]:
H_CISD, E_CISD, CISD_qstate = test.perform_CI(method='CISD', encoding='JW')

In [ ]:
print(E_CISD - pyscf_obj.pyscf_cisd.e_tot)

In [ ]:
# H_CIS, E_CIS, CIS_qstate = test.perform_CI(method='CIS', encoding='JW')
# H_CID, E_CID, CID_qstate = test.perform_CI(method='CID', encoding='JW')
# H_CISD, E_CISD, CISD_qstate = test.perform_CI(method='CISD', encoding='JW')

In [ ]:
print(E_CIS - pyscf_obj.pyscf_hf.e_tot)
print(E_CID - pyscf_obj.pyscf_hf.e_tot)
print(E_CISD - pyscf_obj.pyscf_hf.e_tot)

In [ ]:
print(E_CISD - pyscf_obj.pyscf_cisd.e_tot)

In [ ]:
len(H_CI.nonzero()[0])

In [ ]:
np.isclose((ci_qstate.dagger * H_symmer * ci_qstate).real,
          E_CISD)

In [ ]:
np.isclose(pyscf_obj.pyscf_cisd.e_tot ,
          E_CISD)

In [ ]:
pyscf_obj.pyscf_hf.e_tot

In [ ]:
ci.CISD
pyscf_obj.pyscf_hf

In [ ]:

E_CISD, vec_CISD = eigsh(CI, which='SA', k=1)
E_CISD

In [ ]:
pyscf_obj.pyscf_cisd.e_tot 

In [ ]:
i_rows, j_rows = CI.nonzero()

ind = 40
print(CI[i_rows[ind], j_rows[ind]])
print(CI[j_rows[ind], i_rows[ind]])

In [ ]:
n_occ = H_ferm.n_electrons
n_virt = H_ferm.n_qubits-H_ferm.n_electrons

HF_term = 1
single_terms =  n_occ* n_virt
double_terms = (n_occ**2-n_occ)/2 *  (n_virt**2-n_virt)/2

total = HF_term + single_terms + double_terms
total

In [ ]:
row = np.array([0, 0, 1, 2, 2, 2])

col = np.array([0, 2, 2, 0, 1, 2])

data = np.array([1, 2, 3, 4, 5, 6])

csr_matrix((data, (row, col)), shape=(3, 3)).toarray()

In [ ]:
# https://github.com/psi4/psi4numpy/blob/master/Tutorials/09_Configuration_Interaction/9a_cis.ipynb
# szabo page 236-237

# Build the possible excitations, collect indices into a list
single_dets = []
single_excitations = []
for i in range(H_ferm.n_electrons):
    for a in range(H_ferm.n_electrons, H_ferm.n_qubits):
        single_excitations.append((i, a))
        
        det = hf_state.copy()
        det[[i,a]] = det[[a,i]]
        single_dets.append(det)
# single_excitations


double_dets = []
double_excitations = []
for i in range(H_ferm.n_electrons):
    for j in range(i+1, H_ferm.n_electrons):
        for a in range(H_ferm.n_electrons, H_ferm.n_qubits):
            for b in range(a+1, H_ferm.n_qubits):
                double_excitations.append((i,j, a,b))

                det = hf_state.copy()
                det[[i,a]] = det[[a,i]]
                det[[j,b]] = det[[b,j]]
                double_dets.append(det)

## should calc integrals needed from this!!!

In [ ]:
# single_excitations

In [ ]:
24**2

In [ ]:
print(len(double_dets))
print(len(single_dets))

In [ ]:
# single_excitations

In [ ]:
## scalings:

n_occ = H_ferm.n_electrons
n_virt = H_ferm.n_qubits-H_ferm.n_electrons

print(len(single_excitations) == n_occ* n_virt)
print(len(double_excitations)== (n_occ**2-n_occ)/2 *  (n_virt**2-n_virt)/2)

$$singles = n_{occ} \cdot n_{virt}$$

$$doubles = \bigg( \frac{n_{occ}^{2}-n_{occ}}{2} \cdot \frac{n_{virt}^{2}-n_{virt}}{2} \bigg) $$
$$doubles = \frac{1}{4} (n_{occ}^{2} n_{virt}^{2} - n_{occ}^{2}n_{virt} - n_{occ}n_{virt}^{2} + n_{occ}n_{virt})$$

In [ ]:
from openfermion import FermionOperator, get_sparse_operator


# get_sparse_operator(FermionOperator(f'{2}^ {2}', 1)).toarray()
# get_sparse_operator(FermionOperator(f'{1} {1} {0}^ {0}^', 1)).toarray()

In [ ]:
HF_index = int(''.join(hf_state.astype(str)),2)
H_symmer.to_sparse_matrix[HF_index,HF_index]

In [ ]:
HF_qstate = QuantumState(hf_state.reshape([1,-1]), [1])
HF_qstate.dagger * H_symmer * HF_qstate

In [ ]:
H_ferm.scf_method.e_tot

In [ ]:
#### note this slices QUBIT H_SYMMER

H_red = coo_matrix(H_symmer.to_sparse_matrix.shape,
            dtype=complex
        )
H_red = csr_matrix(H_red)

allowed_dets = [hf_state, *single_dets,*double_dets]
print(len(allowed_dets))

for det_i in allowed_dets:
    for det_j in allowed_dets:
        index_i = int(''.join(det_i.astype(str)),2)
        index_j = int(''.join(det_j.astype(str)),2)
        H_red[index_i, index_j] = H_symmer.to_sparse_matrix[index_i, index_j]

In [ ]:
from scipy.sparse.linalg import eigsh
E_CISD, vec_CISD = eigsh(H_red, which='SA', k=1)
E_CISD

In [ ]:
#### note this slices FERMIONIC!!!!!!! 
H_second = get_sparse_operator(H_ferm.fermionic_molecular_hamiltonian)

H_red = coo_matrix(H_second.shape,
            dtype=complex
        )
H_red = csr_matrix(H_red)

allowed_dets = [hf_state, *single_dets,*double_dets]
print(len(allowed_dets))

for det_i in allowed_dets:
    for det_j in allowed_dets:
        index_i = int(''.join(det_i.astype(str)),2)
        index_j = int(''.join(det_j.astype(str)),2)
        H_red[index_i, index_j] = H_second[index_i, index_j]
E_CISD, vec_CISD = eigsh(H_red, which='SA', k=1)
E_CISD

In [ ]:
# dir(H_ferm)
pyscf_obj.pyscf_cisd.e_tot

In [ ]:
from symmer.symplectic import QuantumState
psi_cisd = QuantumState.from_array(vec_CISD)

In [ ]:
# OVERLAP with HF
psi_cisd.dagger * QuantumState([hf_state])

In [ ]:
psi_cisd.cleanup(zero_threshold=1e-2)

In [ ]:
def sort_det(array):
    """
    
    """

    arr = np.asarray(array)
    n_sites = arr.shape[0]
    sign_dict = {0: +1, 1:-1}
    # Traverse through all array elements
    swap_counter = 0
    for i in range(n_sites):
        swapped = False
        for j in range(0, n_sites - i - 1):
            if arr[j] > arr[j + 1]:
                arr[j], arr[j + 1] = arr[j + 1], arr[j]
                swapped = True
                swap_counter+=1

        if swapped == False:
            break

#     return np.abs(arr.tolist()), sign_dict[swap_counter%2]
    return sign_dict[swap_counter%2]



def get_sign(i_det, j_det, bit_differ):
    
    # first put unique part to start of list (count if swap needed!)
    # then order RHS that contains common elements!
    
    nonzero_i = i_det.nonzero()[0]
    nonzero_j = j_det.nonzero()[0]
    
    unique = bit_differ.nonzero()[0]
    i_unique = np.intersect1d(nonzero_i, unique)
    j_unique = np.intersect1d(nonzero_j, unique)
    
    count_i = 0
    count_j = 0
    for ind, unique_i in enumerate(i_unique):
        swap_ind_i = np.where(unique_i==nonzero_i)[0]
        
        unique_j = j_unique[ind]
        swap_ind_j = np.where(unique_j==nonzero_j)[0]
        
        # swap 
        if ind != swap_ind_i:
            count_i+=1
            nonzero_i[ind], nonzero_i[swap_ind_i] = nonzero_i[swap_ind_i], nonzero_i[ind]
        if ind != swap_ind_j:
            count_j +=1
            nonzero_j[ind], nonzero_j[swap_ind_j] = nonzero_j[swap_ind_j], nonzero_j[ind]
    
    sign_i = sort_det(nonzero_i[len(i_unique):])
    sign_j =  sort_det(nonzero_j[len(j_unique):])
    return sign_i * sign_j * (-1)**(count_i+ count_j)

In [ ]:
from pyscf import ao2mo, gto, scf, ci

full_system_mol = gto.Mole(atom=xyz_file[3:],
                           basis=basis,
                           charge=0,
                           spin=0
                           )
full_system_mol.build()

H_core_ao = scf.hf.get_hcore(full_system_mol)
eri_ao = full_system_mol.intor('int2e')  # 2e- electron repulsion integrals in AO basis

n_docc = full_system_mol.nelectron // 2  # number of double occupied orbitals
n_bas_ft = full_system_mol.nao


SCF_obj = scf.RHF(full_system_mol)
SCF_obj.verbose = 1
SCF_obj.kernel()

SCF_obj.kernel()

pyscf_cisd = ci.CISD(SCF_obj)
pyscf_cisd.verbose = 1
pyscf_cisd.kernel()[0]
pyscf_cisd.e_tot

In [ ]:
from pyscf import ao2mo
# pyscf version!
C = pyscf_obj.pyscf_hf.mo_coeff
H_core_ao = pyscf_obj.pyscf_hf.get_hcore()
n_bas_ft =pyscf_obj.pyscf_hf.mol.nao

pyscf_mo_ints = ao2mo.kernel(pyscf_obj.pyscf_hf.mol, C)

# Convert the 2e integrals (in Chemist’s notation)
pyscf_mo_ints = ao2mo.restore(1, pyscf_mo_ints, pyscf_obj.pyscf_hf.mol.nao)

# pyscf_mo_ints = pyscf_mo_ints.transpose(0,2,3,1) #<- converts to physists notation if needed

# h_ij MO basis
hcore_ij = C.conj().T @ H_core_ao @ C  # # NOT FOCK

###### next convert into spatial to spin
n_spatial = n_bas_ft  # number of atomic orbitals
n_spin = 2 * n_spatial  # two times for spin up and down!

# H_CORE
h_core_spin = np.zeros((n_spin, n_spin))
for p in range(n_spatial):
    for q in range(n_spatial):
        # even indices are SPIN UP!
        h_core_spin[2 * p,
                    2 * q] = hcore_ij[p, q]

        # odd indices are SPIN DOWN!
        h_core_spin[2 * p + 1,
                    2 * q + 1] = hcore_ij[p, q]

# ERI
eri_spin_physicists = np.zeros((n_spin, n_spin, n_spin, n_spin))
eri_mo_spatial_physicists = np.einsum('ijkl -> ikjl', pyscf_mo_ints)
for p in range(n_spatial):
    for q in range(n_spatial):
        for r in range(n_spatial):
            for s in range(n_spatial):
                MO_spatial_term = eri_mo_spatial_physicists[p, q, r, s]

                # up, up, up, up
                eri_spin_physicists[2 * p, 2 * q,
                                    2 * r, 2 * s] = MO_spatial_term

                # down, down, down, down
                eri_spin_physicists[2 * p + 1, 2 * q + 1,
                                    2 * r + 1, 2 * s + 1] = MO_spatial_term

                # up down up down
                eri_spin_physicists[2 * p, 2 * q + 1,
                                    2 * r, 2 * s + 1] = MO_spatial_term

                # down up down up
                eri_spin_physicists[2 * p + 1, 2 * q,
                                    2 * r + 1, 2 * s] = MO_spatial_term

                # other terms go to ZERO!

## physicists notation
# eri_spin_physicists = np.einsum('ikjl -> ijkl', eri_spin_physicists)
# eri_spin_physicists = np.einsum('ikjl -> ijlk', eri_spin_physicists)
CONSTANT_nuclear_energy = full_system_mol.energy_nuc()

h_pq = h_core_spin


H_ci = coo_matrix((2**H_symmer.n_qubits,2**H_symmer.n_qubits) ,
            dtype=complex)
H_ci = csr_matrix(H_ci)

allowed_dets = [hf_state, *single_dets,*double_dets]
for det_i in allowed_dets:
    for det_j in allowed_dets:
        
        bit_diff = np.logical_xor(det_i,det_j)
        n_diff = int(sum(bit_diff))
        
        if n_diff>4:
            pass
        else:
            index_i = int(''.join(det_i.astype(str)),2)
            index_j = int(''.join(det_j.astype(str)),2)
            
    
            mat_element = 0
            if n_diff == 0:
                # <i | H | i>
                occ_inds_i = np.where(det_i)[0]
                for i in occ_inds_i:
                    mat_element+= h_pq[i,i] 
                    
                for ind1, i in enumerate(occ_inds_i[:-1]):
                    for ind2 in range(ind1+1, len(occ_inds_i)):
                        j = occ_inds_i[ind2]
                        mat_element+= (eri_spin_physicists[i,j,i,j] - eri_spin_physicists[i,j,j,i])
                sign = 1
                
            elif n_diff==2:                
                # order matters!
                k = np.logical_and(det_i, bit_diff).nonzero()[0]
                l = np.logical_and(det_j, bit_diff).nonzero()[0]
                mat_element += h_pq[k,l]
                
                common_bits = np.where(np.logical_and(det_i,det_j))[0]
                for i in common_bits:
                    mat_element+= (eri_spin_physicists[k,i,l,i] 
                                   - eri_spin_physicists[k,i,i,l]) 
                
                sign = get_sign(det_i, det_j, bit_diff)
                
            elif n_diff==4:
                ij = np.logical_and(det_i, bit_diff).nonzero()[0]
                kl = np.logical_and(det_j, bit_diff).nonzero()[0]
                i, j = ij[0],ij[1]
                k, l = kl[0], kl[1]
                mat_element += (eri_spin_physicists[i,j,k,l] - eri_spin_physicists[i,j,l,k])
                
                sign = get_sign(det_i, det_j, bit_diff)
                
                
            
            H_ci[index_i, index_j] = mat_element * sign

In [ ]:
E_CISD, vec_CISD = eigsh(H_ci, which='SA', k=1)
E_CISD + SCF_obj.energy_nuc()

In [ ]:
pyscf_obj.pyscf_cisd.e_tot

In [ ]:
(E_CISD + SCF_obj.energy_nuc()) - pyscf_obj.pyscf_cisd.e_tot

In [ ]:
# out = H_red - H_ci
# np.einsum('ij->', out.toarray())

In [ ]:
np.allclose(abs(H_red.toarray()),
            abs(H_ci.toarray()))

In [ ]:
np.allclose(H_red.toarray(),
            H_ci.toarray())

In [ ]:
### Difference could be due to re-ordering with signs
### as opposed to just slicing the element (aka no sign changes)

### still gives correct answer

In [ ]:
done

In [ ]:
A = np.array([0,1,1,0])
B = np.array([1,1,1,1])
sum(np.logical_xor(A,B))

In [ ]:
# ecisd, civec = pyscf_obj.pyscf_cisd.kernel()

# full_cisd_vec = pyscf_obj.pyscf_cisd.to_fcivec(civec, 
#                                           pyscf_obj.pyscf_cisd.mol.nao,
#                                           pyscf_obj.pyscf_cisd.mol.nelectron
#                                           )

# full_cisd_vec.shape

In [ ]:
# single_excitations, double_excitations
# single_excitations
# double_excitations

In [ ]:
h_pq.shape

In [ ]:
H_ferm.fermionic_fock_operator

In [ ]:
# A = np.array([1,1,0,0,1,1,0,0])
# B = np.array([1,1,1,1,0,0,0,0])

A = np.array([1,1,1,0,0,0,0,1])
B = np.array([1,1,0,1,1,0,0,0])

bit_diff = np.logical_xor(A,B)
bit_diff.astype(int)

np.logical_and(A, bit_diff).nonzero()

In [ ]:
# https://iopscience.iop.org/article/10.1088/2058-9565/aa9463/pdf

from openfermion.chem.molecular_data import spinorb_from_spatial

h_pq, eri_spin_physicists = spinorb_from_spatial(
    H_ferm._one_body_integrals,  H_ferm._two_body_integrals)

# eri_spin_physicists = np.einsum('ikjl -> ijkl', eri_spin_physicists)
# eri_spin_physicists = np.einsum('ijkl -> ikjl', eri_spin_physicists)
eri_mo_spatial_physicists = np.einsum('ikjl -> ijkl', pyscf_mo_ints)
H_ci = coo_matrix((2**H_symmer.n_qubits,2**H_symmer.n_qubits) ,
            dtype=complex
        )
H_ci = csr_matrix(H_ci)

allowed_dets = [hf_state, *single_dets,*double_dets]
for det_i in allowed_dets:
    for det_j in allowed_dets:
        
        bit_diff = np.logical_xor(det_i,det_j)
        n_diff = int(sum(bit_diff))
        
        if n_diff>4:
            pass
        else:
            index_i = int(''.join(det_i.astype(str)),2)
            index_j = int(''.join(det_j.astype(str)),2)
            mat_element = 0
            
            if n_diff == 0:
                # <i | H | i>
                occ_inds_i = np.where(det_i)[0]
                for i in occ_inds_i:
                    mat_element+= h_pq[i,i]
                    
                for ind1, i in enumerate(occ_inds_i[:-1]):
                    for ind2 in range(ind1+1, len(occ_inds_i)):
                        j = occ_inds_i[ind2]
                        mat_element+= eri_spin_physicists[i,j,i,j] - eri_spin_physicists[i,j,j,i]

            elif n_diff==2:                
#                 diff_bit_indices = np.where(bit_diff)[0]
#                 k = diff_bit_indices[0]
#                 l = diff_bit_indices[1]
                # order matters!
                k = np.logical_and(det_i, bit_diff).nonzero()[0]
                l = np.logical_and(det_j, bit_diff).nonzero()[0]
                mat_element += h_pq[k,l]
                
                common_bits = np.where(np.logical_and(det_i,det_j))[0]
                for i in common_bits:
                    mat_element+= (eri_spin_physicists[k,i,l,i] 
                                   - eri_spin_physicists[k,i,i,l])
                
            elif n_diff==4:
#                 diff_bit_indices = np.where(bit_diff)[0]
#                 i = diff_bit_indices[0]
#                 j =diff_bit_indices[1]
#                 k = diff_bit_indices[2]
#                 l = diff_bit_indices[3]
                ij = np.logical_and(det_i, bit_diff).nonzero()[0]
                kl = np.logical_and(det_j, bit_diff).nonzero()[0]
                i, j = ij[0],ij[1]
                k, l = kl[0], kl[1]
                mat_element += eri_spin_physicists[i,j,k,l] - eri_spin_physicists[i,j,l,k]
            
            H_ci[index_i, index_j] = mat_element

In [ ]:
kl

In [ ]:
E_CISD, vec_CISD = eigsh(H_ci, which='SA', k=1)
E_CISD + pyscf_obj.pyscf_hf.energy_nuc()

In [ ]:
pyscf_obj.pyscf_cisd.e_corr

In [ ]:
# pyscf_obj.pyscf_hf.energy_elec()[0] 

In [ ]:
pyscf_obj.pyscf_cisd.e_tot

In [ ]:
H_ci.shape

In [ ]:
out = H_red - H_ci

np.einsum('ij->', out.toarray())

In [ ]:
from pyscf import ao2mo, gto, scf, ci

full_system_mol = gto.Mole(atom=xyz_file[3:],
                           basis=basis,
                           charge=0,
                           spin=0
                           )
full_system_mol.build()

H_core_ao = scf.hf.get_hcore(full_system_mol)
eri_ao = full_system_mol.intor('int2e')  # 2e- electron repulsion integrals in AO basis

n_docc = full_system_mol.nelectron // 2  # number of double occupied orbitals
n_bas_ft = full_system_mol.nao


SCF_obj = scf.RHF(full_system_mol)
SCF_obj.verbose = 1
SCF_obj.kernel()

SCF_obj.kernel()

pyscf_cisd = ci.CISD(SCF_obj)
pyscf_cisd.verbose = 1
pyscf_cisd.kernel()[0]
pyscf_cisd.e_tot

In [ ]:
from pyscf import ao2mo
# pyscf version!
C = pyscf_obj.pyscf_hf.mo_coeff
H_core_ao = pyscf_obj.pyscf_hf.get_hcore()
n_bas_ft =pyscf_obj.pyscf_hf.mol.nao

pyscf_mo_ints = ao2mo.kernel(pyscf_obj.pyscf_hf.mol, C)

# Convert the 2e integrals (in Chemist’s notation)
pyscf_mo_ints = ao2mo.restore(1, pyscf_mo_ints, pyscf_obj.pyscf_hf.mol.nao)

# pyscf_mo_ints = pyscf_mo_ints.transpose(0,2,3,1) #<- converts to physists notation if needed

# h_ij MO basis
hcore_ij = C.conj().T @ H_core_ao @ C  # # NOT FOCK

###### next convert into spatial to spin
n_spatial = n_bas_ft  # number of atomic orbitals
n_spin = 2 * n_spatial  # two times for spin up and down!

# H_CORE
h_core_spin = np.zeros((n_spin, n_spin))
for p in range(n_spatial):
    for q in range(n_spatial):
        # even indices are SPIN UP!
        h_core_spin[2 * p,
                    2 * q] = hcore_ij[p, q]

        # odd indices are SPIN DOWN!
        h_core_spin[2 * p + 1,
                    2 * q + 1] = hcore_ij[p, q]

# ERI
eri_spin_physicists = np.zeros((n_spin, n_spin, n_spin, n_spin))
eri_mo_spatial_physicists = np.einsum('ijkl -> ikjl', pyscf_mo_ints)
for p in range(n_spatial):
    for q in range(n_spatial):
        for r in range(n_spatial):
            for s in range(n_spatial):
                MO_spatial_term = eri_mo_spatial_physicists[p, q, r, s]

                # up, up, up, up
                eri_spin_physicists[2 * p, 2 * q,
                                    2 * r, 2 * s] = MO_spatial_term

                # down, down, down, down
                eri_spin_physicists[2 * p + 1, 2 * q + 1,
                                    2 * r + 1, 2 * s + 1] = MO_spatial_term

                # up down up down
                eri_spin_physicists[2 * p, 2 * q + 1,
                                    2 * r, 2 * s + 1] = MO_spatial_term

                # down up down up
                eri_spin_physicists[2 * p + 1, 2 * q,
                                    2 * r + 1, 2 * s] = MO_spatial_term

                # other terms go to ZERO!

## physicists notation
# eri_spin_physicists = np.einsum('ikjl -> ijkl', eri_spin_physicists)
# eri_spin_physicists = np.einsum('ikjl -> ijlk', eri_spin_physicists)
CONSTANT_nuclear_energy = full_system_mol.energy_nuc()

h_pq = h_core_spin


H_ci = coo_matrix((2**H_symmer.n_qubits,2**H_symmer.n_qubits) ,
            dtype=complex)
H_ci = csr_matrix(H_ci)

allowed_dets = [hf_state, *single_dets,*double_dets]
for det_i in allowed_dets:
    for det_j in allowed_dets:
        
        bit_diff = np.logical_xor(det_i,det_j)
        n_diff = int(sum(bit_diff))
        
        if n_diff>4:
            pass
        else:
            index_i = int(''.join(det_i.astype(str)),2)
            index_j = int(''.join(det_j.astype(str)),2)
            mat_element = 0
            
            if n_diff == 0:
                # <i | H | i>
                occ_inds_i = np.where(det_i)[0]
                for i in occ_inds_i:
                    mat_element+= h_pq[i,i]
                    
                for ind1, i in enumerate(occ_inds_i[:-1]):
                    for ind2 in range(ind1+1, len(occ_inds_i)):
                        j = occ_inds_i[ind2]
                        mat_element+= eri_spin_physicists[i,j,i,j] - eri_spin_physicists[i,j,j,i]

            elif n_diff==2:                
#                 diff_bit_indices = np.where(bit_diff)[0]
#                 k = diff_bit_indices[0]
#                 l = diff_bit_indices[1]
                # order matters!
                k = np.logical_and(det_i, bit_diff).nonzero()[0]
                l = np.logical_and(det_j, bit_diff).nonzero()[0]
                mat_element += h_pq[k,l]
                
                common_bits = np.where(np.logical_and(det_i,det_j))[0]
                for i in common_bits:
                    mat_element+= (eri_spin_physicists[k,i,l,i] 
                                   - eri_spin_physicists[k,i,i,l])
                
            elif n_diff==4:
#                 diff_bit_indices = np.where(bit_diff)[0]
#                 i = diff_bit_indices[0]
#                 j =diff_bit_indices[1]
#                 k = diff_bit_indices[2]
#                 l = diff_bit_indices[3]
                ij = np.logical_and(det_i, bit_diff).nonzero()[0]
                kl = np.logical_and(det_j, bit_diff).nonzero()[0]
                i, j = ij[0],ij[1]
                k, l = kl[0], kl[1]
                mat_element += eri_spin_physicists[i,j,k,l] - eri_spin_physicists[i,j,l,k]
            
            
            sign = 1
            alphaPositions = np.where(det_i[::2])[0] *2 
            betaPositions = np.where(det_i[1::2])[0] *2  +1
            for i in range(len(alphaPositions)):
                if (alphaPositions[i] - i) % 2 == 1:
                    sign = -sign
            for i in range(len(betaPositions)):
                if (betaPositions[i] - i) % 2 == 1:
                    sign = -sign
            
            alphaPositions = np.where(det_j[::2])[0] *2 
            betaPositions = np.where(det_j[1::2])[0] *2  +1
            for i in range(len(alphaPositions)):
                if (alphaPositions[i] - i) % 2 == 1:
                    sign = -sign
            for i in range(len(betaPositions)):
                if (betaPositions[i] - i) % 2 == 1:
                    sign = -sign
            H_ci[index_i, index_j] = mat_element * sign

In [ ]:
E_CISD, vec_CISD = eigsh(H_ci, which='SA', k=1)
E_CISD + SCF_obj.energy_nuc()

In [ ]:
pyscf_cisd.e_tot

In [ ]:
np.allclose(H_red.toarray(),
            H_ci.toarray())

In [ ]:
H1 = H_red.toarray()
H2 = H_ci.toarray()
diff = (H1 - H2)
np.einsum('ij->', diff)

diff_h = abs(H1 - H2)
diff_inds = np.where(diff_h>1e-12)

# ind=6$
ind=12
row = diff_inds[0][ind]
col = diff_inds[1][ind]
print('wrong:', len(diff_inds[0]))
print(row,col)

print(H1[row, col] )
print(H2[row, col])

print(np.binary_repr(row, width=H_ferm.n_qubits))
print(np.binary_repr(col, width=H_ferm.n_qubits))

In [ ]:
T = np.array(list(np.binary_repr(row, width=H_ferm.n_qubits))).astype(int)
W = np.array(list(np.binary_repr(col, width=H_ferm.n_qubits))).astype(int)
# sort_det(T) * sort_det(W) * sort_det(hf_state)
# sort_det(W)

In [ ]:
# print(W)
# print(hf_state)
# print(np.logical_and(hf_state,W).astype(int))


In [ ]:
T = np.array([0,1,0,1,0,1,0,1])
W = np.array([0,0,1,1,1,1,0,0])

# T = np.array([0,0,1,1,0,0])
# W = np.array([0,0,0,1,0,1])

print('T', T)
nonz_T = T.nonzero()[0]
print('T', nonz_T)
print()
print('W', W)
nonz_W = W.nonzero()[0]
print('W', nonz_W)

common_ints = np.intersect1d(nonz_T, nonz_W)
print('common', common_ints)

T_to_sort = [i if i in common_ints else 0 for i in nonz_T ]
W_to_sort = [i if i in common_ints else 0 for i in nonz_W ]

print(T_to_sort)
print(W_to_sort)


sort_det(T_to_sort)
# T_mod = T.copy()
# for i in nonz_T:
#     if i in common_ints:
#         T_mod[i]=1
# T_mod


In [ ]:
sort_det(hf_state)
hf_state
np.log

In [ ]:
HF_index

In [ ]:
np.allclose(abs(H1),
            abs(H2))

In [ ]:
np.logical_and(3,2)

In [ ]:

np.logical_and(W,T)

In [ ]:
print(W)
print(T)


In [ ]:
sign=1
temp = T+W
for i in range(len(temp)):
    if (temp[i] - i) % 2 == 1:
        sign = -sign
sign

In [ ]:
def sort_det(array):
    """
    
    """

    arr = np.asarray(array)*-1
    n_sites = arr.shape[0]
    sign_dict = {0: +1, 1:-1}
    # Traverse through all array elements
    swap_counter = 0
    for i in range(n_sites):
        swapped = False
        for j in range(0, n_sites - i - 1):
            if arr[j] > arr[j + 1]:
                arr[j], arr[j + 1] = arr[j + 1], arr[j]
                swapped = True
                swap_counter+=1

        if swapped == False:
            break

#     return np.abs(arr.tolist()), sign_dict[swap_counter%2]
    return sign_dict[swap_counter%2]

sort_det([0,1,1,1])

In [ ]:
A=[1,0,1,0]
B=[1,0,0,1]
# sort_det(A) * sort_det(B)
sort_det(np.logical_or(A,B).astype(int))

In [ ]:
W_temp = np.arange(len(W.nonzero()[0]))
# W_temp[W.nonzero()]*=-1
# bubble_sort_maj(W_temp)
W_temp

In [ ]:


bit_diff = np.logical_xor(det_i,det_j)
# k = np.logical_and(det_i, bit_diff).nonzero()[0]
# l = np.logical_and(det_j, bit_diff).nonzero()[0]

ij = np.logical_and(det_i, bit_diff).nonzero()[0]
kl = np.logical_and(det_j, bit_diff).nonzero()[0]
i, j = ij[0],ij[1]
k, l = kl[0], kl[1]

-1**(sum(det_i[:(i-1)]) + sum(det_i[:(j-1)])) * -1**(sum(det_j[:(k-1)]) + sum(det_j[:(l-1)]))

In [ ]:
sign = 1
alphaPositions = np.where(det_i[::2])[0]
alphaPositions = alphaPositions*2

betaPositions = np.where(det_i[1::2])[0]
betaPositions = betaPositions*2 + 1

sign=1
for i in range(len(alphaPositions)):
    if (alphaPositions[i] - i) % 2 == 1:
        sign = -sign
for i in range(len(betaPositions)):
    if (betaPositions[i] - i) % 2 == 1:
        sign = -sign
sign

In [ ]:
T = np.array(list(np.binary_repr(row, width=H_ferm.n_qubits))).astype(int)
W = np.array(list(np.binary_repr(col, width=H_ferm.n_qubits))).astype(int)
bit_diff = np.logical_xor(W,T)
# bit_diff
permutations = sum(bit_diff.nonzero()[0])/2
-1**permutations

In [ ]:
# alphaPositions, betaPositions = self.getOrbitalPositionLists(alphaIndexList, betaIndexList)
#         for i in range(len(alphaPositions)):
#             if (alphaPositions[i] - i) % 2 == 1:
#                 sign = -sign
#         for i in range(len(betaPositions)):
#             if (betaPositions[i] - i) % 2 == 1:
#                 sign = -sign
#         return sign
    

# np.where()[0]


In [ ]:
# def bubble_sort_maj(array):
#     """

#     order a determinant so occ first
#     Count number of swaps done
#     """

#     arr = np.asarray(array)
#     n_sites = arr.shape[0]
#     sign_dict = {0: +1, 1:-1}
#     # Traverse through all array elements
#     swap_counter = 0
#     for i in range(n_sites):
#         swapped = False
#         for j in range(0, n_sites - i - 1):
#             if arr[j] > arr[j + 1]:
#                 arr[j], arr[j + 1] = arr[j + 1], arr[j]
#                 swapped = True
#                 swap_counter+=1

#         if swapped == False:
#             break

#     return arr.tolist(), sign_dict[swap_counter%2]

In [ ]:
B

In [ ]:
np.setdiff1d([1,2,3], 1)

In [ ]:
from openfermion.chem.molecular_data import spinorb_from_spatial

h_pq, eri_spin_physicists = spinorb_from_spatial(
    H_ferm._one_body_integrals,  H_ferm._two_body_integrals
)



CONSTANT_nuclear_energy = pyscf_obj.pyscf_hf.mol.energy_nuc()
fermionic_CISD = FermionOperator('', CONSTANT_nuclear_energy)
for q,p in single_excitations:
    fermionic_CISD += FermionOperator(f'{p}^ {q}', h_pq[p,q])

for s,q,p,r in double_excitations:
    g_pqrs = eri_spin_physicists[p, q, r, s]
    fermionic_CISD += 0.5 * FermionOperator(f'{p}^ {r}^ {s} {q}', g_pqrs)

    
# fermionic_CISD += H_ferm.fermionic_fock_operator + 
# for p in range(n_spin):
#     for q in range(n_spin):
#         h_pq = h_core_spin[p, q]
#         fermionic_H += FermionOperator(f'{p}^ {q}', h_pq)

#         for r in range(n_spin):
#             for s in range(n_spin):
#                 g_pqrs = eri_spin_physicists[p, q, r, s]
#                 fermionic_H += 0.5 * FermionOperator(f'{p}^ {r}^ {s} {q}', g_pqrs)

In [ ]:
E, vec = eigsh(get_sparse_operator(fermionic_CISD), which='SA', k=1)
E

In [ ]:
RDM_1 = pyscf_obj.pyscf_cisd.make_rdm1()

# spin_RDM_1 = np.zeros((RDM_1.shape[0]*2, RDM_1.shape[1]*2))



single_terms = FermionOperator()
for i, a in single_excitations:
    alpha = RDM_1[i//2,a//2]
    beta = alpha # restricted!
    op1 = FermionOperator(f'{2*a}^ {2*i}', alpha)
    op2 = FermionOperator(f'{2*a+1}^ {2*i+1}', beta)
    single_terms+= op1
    single_terms += op2

    

In [ ]:
double_excitations[100]

In [ ]:
# could also brute force

In [ ]:
double_dets[50] == double_dets[2]

In [ ]:
hf_state = H_ferm.hf_comp_basis_state
hf_state


In [ ]:
hf_ket = H_ferm.hf_ket
hf_ket.shape

In [ ]:
hfock_energy = pyscf_obj.pyscf_hf.energy_tot()
print(hfock_energy)

H_mat = H_ferm.get_sparse_ham()
hf_ket.conj().T @ H_mat @ hf_ket

In [ ]:
from symmer.chemistry import Draw_cube_orbital

print('indices:', list(range(pyscf_obj.pyscf_hf.mo_coeff.shape[1])))

index_list = [3,4]

orb_list = Draw_cube_orbital(pyscf_obj.pyscf_hf.mol,
                           xyz_file,  
                           pyscf_obj.pyscf_hf.mo_coeff,
                           index_list,
                           width=400,
                           height=400,
                           style="stick") # change to sphere/stick
for orb in orb_list:
    orb.show()

In [ ]:
get_sparse_operator( FermionOperator(f'{1}^ {1}', 1)).toarray()

In [ ]:
get_sparse_operator( FermionOperator(f'{0}^ {1}', 1), n_qubits=2).toarray()

In [ ]:
get_sparse_operator( FermionOperator(f'{0}^ {1}^ {1} {0}', 1), n_qubits=2).toarray()

In [ ]:
get_sparse_operator( FermionOperator(f'{1}^ {0}^ {1} {1}', 1), n_qubits=2).toarray()

In [ ]:



    

sort_det([4,5,2])

In [ ]:
diff = np.logical_xor(T,W)
print(T)
print(W)
get_sign(T,W,diff)

In [ ]:
# T = np.array([0,1,0,1,0,1,0,1])
# W = np.array([0,0,1,1,1,1,0,0])

diff = np.logical_xor(T,W)
get_sign(T,W, diff)

In [ ]:
# T = np.array([0,1,0,1,0,1,0,1])
# W = np.array([0,0,1,1,1,1,0,0])

# T = np.array([0,0,1,1,0,0])
# W = np.array([0,0,0,1,0,1])

print('T', T)
nonz_T = T.nonzero()[0]
print('T', nonz_T)
print()
print('W', W)
nonz_W = W.nonzero()[0]
print('W', nonz_W)

diff = np.logical_xor(T,W)
unique = diff.nonzero()[0]
T_unique = np.intersect1d(nonz_T, unique)
W_unique = np.intersect1d(nonz_W, unique)


for ind, t in enumerate(T_unique):
    swap_ind = np.where(t==nonz_T)[0]
    nonz_T[ind], nonz_T[swap_ind] = nonz_T[swap_ind], nonz_T[ind]

for ind, w in enumerate(W_unique):
    swap_ind = np.where(w==nonz_W)[0]
    nonz_W[ind], nonz_W[swap_ind] = nonz_W[swap_ind], nonz_W[ind]

print(nonz_W)
print(nonz_T) 

sign = sort_det(nonz_W[len(T_unique):]) * sort_det(nonz_T[len(T_unique):])
print(sign)
    # nonz_T[1], nonz_T[T_unique[1]] = nonz_T[T_unique[1]], nonz_T[1]
# common_ints = np.intersect1d(nonz_T, nonz_W)
# print('common', common_ints)

# T_to_sort = [i if i in common_ints else 0 for i in nonz_T ]
# W_to_sort = [i if i in common_ints else 0 for i in nonz_W ]

# print(T_to_sort)
# print(W_to_sort)


# sort_det(T_to_sort)
# # T_mod = T.copy()
# # for i in nonz_T:
# #     if i in common_ints:
# #         T_mod[i]=1
# # T_mod


In [ ]:
def sort_det(array):
    """
    
    """

    arr = np.asarray(array)
    n_sites = arr.shape[0]
    sign_dict = {0: +1, 1:-1}
    # Traverse through all array elements
    swap_counter = 0
    for i in range(n_sites):
        swapped = False
        for j in range(0, n_sites - i - 1):
            if arr[j] > arr[j + 1]:
                arr[j], arr[j + 1] = arr[j + 1], arr[j]
                swapped = True
                swap_counter+=1

        if swapped == False:
            break

#     return np.abs(arr.tolist()), sign_dict[swap_counter%2]
    return sign_dict[swap_counter%2]



def get_sign(i_det, j_det, bit_differ):
    
    # first put unique part to start of list (count if swap needed!)
    # then order RHS that contains common elements!
    
    nonzero_i = i_det.nonzero()[0]
    nonzero_j = j_det.nonzero()[0]
    
    unique = bit_differ.nonzero()[0]
    i_unique = np.intersect1d(nonzero_i, unique)
    j_unique = np.intersect1d(nonzero_j, unique)
    
    count_i = 0
    count_j = 0
    for ind, unique_i in enumerate(i_unique):
        swap_ind_i = np.where(unique_i==nonzero_i)[0]
        
        unique_j = j_unique[ind]
        swap_ind_j = np.where(unique_j==nonzero_j)[0]
        
        # swap 
        if ind != swap_ind_i:
            count_i+=1
            nonzero_i[ind], nonzero_i[swap_ind_i] = nonzero_i[swap_ind_i], nonzero_i[ind]
        if ind != swap_ind_j:
            count_j +=1
            nonzero_j[ind], nonzero_j[swap_ind_j] = nonzero_j[swap_ind_j], nonzero_j[ind]
    
    sign_i = sort_det(nonzero_i[len(i_unique):])
    sign_j =  sort_det(nonzero_j[len(j_unique):])
    return sign_i * sign_j * (-1)**(count_i+ count_j)

In [ ]:
from pyscf import ao2mo, gto, scf, ci

full_system_mol = gto.Mole(atom=xyz_file[3:],
                           basis=basis,
                           charge=0,
                           spin=0
                           )
full_system_mol.build()

H_core_ao = scf.hf.get_hcore(full_system_mol)
eri_ao = full_system_mol.intor('int2e')  # 2e- electron repulsion integrals in AO basis

n_docc = full_system_mol.nelectron // 2  # number of double occupied orbitals
n_bas_ft = full_system_mol.nao


SCF_obj = scf.RHF(full_system_mol)
SCF_obj.verbose = 1
SCF_obj.kernel()

SCF_obj.kernel()

pyscf_cisd = ci.CISD(SCF_obj)
pyscf_cisd.verbose = 1
pyscf_cisd.kernel()[0]
pyscf_cisd.e_tot

In [ ]:
from pyscf import ao2mo
# pyscf version!
C = pyscf_obj.pyscf_hf.mo_coeff
H_core_ao = pyscf_obj.pyscf_hf.get_hcore()
n_bas_ft =pyscf_obj.pyscf_hf.mol.nao

pyscf_mo_ints = ao2mo.kernel(pyscf_obj.pyscf_hf.mol, C)

# Convert the 2e integrals (in Chemist’s notation)
pyscf_mo_ints = ao2mo.restore(1, pyscf_mo_ints, pyscf_obj.pyscf_hf.mol.nao)

# pyscf_mo_ints = pyscf_mo_ints.transpose(0,2,3,1) #<- converts to physists notation if needed

# h_ij MO basis
hcore_ij = C.conj().T @ H_core_ao @ C  # # NOT FOCK

###### next convert into spatial to spin
n_spatial = n_bas_ft  # number of atomic orbitals
n_spin = 2 * n_spatial  # two times for spin up and down!

# H_CORE
h_core_spin = np.zeros((n_spin, n_spin))
for p in range(n_spatial):
    for q in range(n_spatial):
        # even indices are SPIN UP!
        h_core_spin[2 * p,
                    2 * q] = hcore_ij[p, q]

        # odd indices are SPIN DOWN!
        h_core_spin[2 * p + 1,
                    2 * q + 1] = hcore_ij[p, q]

# ERI
eri_spin_physicists = np.zeros((n_spin, n_spin, n_spin, n_spin))
eri_mo_spatial_physicists = np.einsum('ijkl -> ikjl', pyscf_mo_ints)
for p in range(n_spatial):
    for q in range(n_spatial):
        for r in range(n_spatial):
            for s in range(n_spatial):
                MO_spatial_term = eri_mo_spatial_physicists[p, q, r, s]

                # up, up, up, up
                eri_spin_physicists[2 * p, 2 * q,
                                    2 * r, 2 * s] = MO_spatial_term

                # down, down, down, down
                eri_spin_physicists[2 * p + 1, 2 * q + 1,
                                    2 * r + 1, 2 * s + 1] = MO_spatial_term

                # up down up down
                eri_spin_physicists[2 * p, 2 * q + 1,
                                    2 * r, 2 * s + 1] = MO_spatial_term

                # down up down up
                eri_spin_physicists[2 * p + 1, 2 * q,
                                    2 * r + 1, 2 * s] = MO_spatial_term

                # other terms go to ZERO!

## physicists notation
# eri_spin_physicists = np.einsum('ikjl -> ijkl', eri_spin_physicists)
# eri_spin_physicists = np.einsum('ikjl -> ijlk', eri_spin_physicists)
CONSTANT_nuclear_energy = full_system_mol.energy_nuc()

h_pq = h_core_spin


In [ ]:
# H_ci = coo_matrix((2**H_symmer.n_qubits,2**H_symmer.n_qubits) ,
#             dtype=complex)
# H_ci = csr_matrix(H_ci)
data =[]
row=[]
col=[]

allowed_dets = [hf_state, *single_dets,*double_dets]
# allowed_dets = [*single_dets]
for det_i in allowed_dets:
    for det_j in allowed_dets:
        
        bit_diff = np.logical_xor(det_i,det_j)
        n_diff = int(sum(bit_diff))
        
        if n_diff>4:
            pass
        else:
            index_i = int(''.join(det_i.astype(str)),2)
            index_j = int(''.join(det_j.astype(str)),2)
            
    
            mat_element = 0
            if n_diff == 0:
                # <i | H | i>
                occ_inds_i = np.where(det_i)[0]
                for i in occ_inds_i:
                    mat_element+= h_pq[i,i]
                    
                for ind1, i in enumerate(occ_inds_i[:-1]):
                    for ind2 in range(ind1+1, len(occ_inds_i)):
                        j = occ_inds_i[ind2]
                        mat_element+= (eri_spin_physicists[i,j,i,j] - eri_spin_physicists[i,j,j,i])
                sign = 1
                
            elif n_diff==2:                
                # order matters!
                k = np.logical_and(det_i, bit_diff).nonzero()[0]
                l = np.logical_and(det_j, bit_diff).nonzero()[0]
                mat_element += h_pq[k,l]
                
                common_bits = np.where(np.logical_and(det_i,det_j))[0]
                for i in common_bits:
                    mat_element+= (eri_spin_physicists[k,i,l,i] 
                                   - eri_spin_physicists[k,i,i,l]) 
                
                sign = get_sign(det_i, det_j, bit_diff)
                
            elif n_diff==4:
                ij = np.logical_and(det_i, bit_diff).nonzero()[0]
                kl = np.logical_and(det_j, bit_diff).nonzero()[0]
                i, j = ij[0],ij[1]
                k, l = kl[0], kl[1]
                mat_element += (eri_spin_physicists[i,j,k,l] - eri_spin_physicists[i,j,l,k])
                
                sign = get_sign(det_i, det_j, bit_diff)
                
                
            
            data.append(float(mat_element * sign))
            row.append(index_i)
            col.append(index_j)

H_ci = csr_matrix((data, (row, col)), shape=(2**(2*n_spatial), 2**(2*n_spatial)))

E_CI, vec_CI = eigsh(H_ci, which='SA', k=1)
ci_qstate = QuantumState.from_array(vec_CI)
del vec_CI

In [ ]:
E_CI - pyscf_cisd.e_tot

In [ ]:
E_CI - SCF_obj.e_tot